##### # Import neccessary modules
1 for matrix caculation, 2 for Deep Learning

In [1]:
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from module.input_parser_old import input_parser
from module import in_fun
from module.pl_module import BinaryClassifier
from module.model import AnnModel
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

beacon_list = [f"beacon{idx + 1}" for idx in range(9)]

%matplotlib widget

# Load Data
python에서 csv파일을 읽어올때, 숫자만 있는것이 아닌 문자열이 포함된 데이터를 읽을 경우 느려지는 것 확인

In [2]:
data_path = "data/train/infra_free/"
room_list = [room + '/' for room in os.listdir(data_path)]
room_data_list = []
for room in room_list:
    file_list = [file for file in os.listdir(data_path + room) if file.endswith(".csv")]
    cur_room_file = []
    for file in file_list:
        cur_room_file.append(pd.read_csv(data_path + room + file, header=None).to_numpy())
    room_data_list.append(np.concatenate((cur_room_file), axis=0))

In [3]:
for room_data in room_data_list:
    print(np.shape(room_data))

(26, 101)
(22, 101)
(22, 101)
(22, 101)


# Data 만들기
같은 방 데이터 끼리 매칭시키고, 다른 방 데이터 끼리 매칭시키고

In [4]:
matched_same_ble = []
for room_data in room_data_list:
    matched_same_ble.append(in_fun.train_matcher(room_data[:,1:41], room_data[:,1:41]))
same_ble = np.concatenate(matched_same_ble)
print(np.shape(same_ble))

(2128, 80)


In [5]:
matched_diff_ble = []
for idx in range(len(room_data_list)):
    if idx == 0:
        matched_diff_ble.append(in_fun.train_matcher(room_data_list[-1][:,1:41], room_data_list[idx][:,1:41]))
    else:
        matched_diff_ble.append(in_fun.train_matcher(room_data_list[idx-1][:,1:41], room_data_list[idx][:,1:41]))
diff_ble = np.concatenate(matched_diff_ble)
print(np.shape(diff_ble))

(2112, 80)


In [6]:
matched_same_wifi = []
for room_data in room_data_list:
    matched_same_wifi.append(in_fun.train_matcher(room_data[:,41:81], room_data[:,41:81]))
same_wifi = np.concatenate(matched_same_wifi)
print(np.shape(same_wifi))

(2128, 80)


In [7]:
matched_diff_wifi = []
for idx in range(len(room_data_list)):
    if idx == 0:
        matched_diff_wifi.append(in_fun.train_matcher(room_data_list[-1][:,41:81], room_data_list[idx][:,41:81]))
    else:
        matched_diff_wifi.append(in_fun.train_matcher(room_data_list[idx-1][:,41:81], room_data_list[idx][:,41:81]))
    
diff_wifi = np.concatenate(matched_diff_wifi)
print(np.shape(diff_wifi))

(2112, 80)


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:0


In [9]:
print(len(same_ble) / (len(same_ble) + len(diff_ble)))

0.5018867924528302


# Loading data for nn

In [10]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

x_tensor_ble = torch.from_numpy(np.concatenate((same_ble, diff_ble))).float()

y = np.concatenate((np.zeros([len(same_ble), 1]) + 1, np.zeros([len(diff_ble), 1])))
y_tensor = torch.from_numpy(y).float()

dataset_ble = TensorDataset(x_tensor_ble, y_tensor)

train_len_ble = len(dataset_ble) // 5 * 4
val_len_ble  = len(dataset_ble) - train_len_ble

train_dataset_ble, val_dataset_ble = random_split(dataset_ble, [train_len_ble, val_len_ble])

train_loader_ble = DataLoader(dataset = train_dataset_ble, batch_size=32, shuffle=True)
val_loader_ble = DataLoader(dataset = val_dataset_ble, batch_size = val_len_ble)

In [11]:
annModel_ble = AnnModel(input_size = 80)
binaryClassifier_ble = BinaryClassifier(annModel_ble)
trainer_ble = pl.Trainer(gpus=1, max_epochs = 100)
trainer_ble.fit(binaryClassifier_ble, train_loader_ble, val_loader_ble)
torch.save(annModel_ble.state_dict(), "./model/ble")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | AnnModel          | 7.3 K 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
7.3 K     Trainable params
0         Non-trainable params
7.3 K     Total params
0.029     Total estimated model params size (MB)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


# Load Wifi data

In [12]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

x_tensor_wifi = torch.from_numpy(np.concatenate((same_wifi, diff_wifi))).float()

y = np.concatenate((np.zeros([len(same_wifi), 1]) + 1, np.zeros([len(diff_wifi), 1])))
y_tensor = torch.from_numpy(y).float()

dataset_wifi = TensorDataset(x_tensor_wifi, y_tensor)

train_len_wifi = len(dataset_wifi) // 5 * 4
val_len_wifi = len(dataset_wifi) - train_len_wifi

train_dataset_wifi, val_dataset_wifi = random_split(dataset_wifi, [train_len_wifi, val_len_wifi])

train_loader_wifi = DataLoader(dataset = train_dataset_wifi, batch_size=32, shuffle=True)
val_loader_wifi = DataLoader(dataset = val_dataset_wifi, batch_size = val_len_wifi)


In [13]:
annModel_wifi = AnnModel(input_size = 80)
binaryClassifier_wifi = BinaryClassifier(annModel_wifi)
trainer_wifi = pl.Trainer(gpus=1, max_epochs = 100)
trainer_wifi.fit(binaryClassifier_wifi, train_loader_wifi, val_loader_wifi)
torch.save(annModel_wifi.state_dict(), "./model/wifi")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | AnnModel          | 7.3 K 
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
7.3 K     Trainable params
0         Non-trainable params
7.3 K     Total params
0.029     Total estimated model params size (MB)
